In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
from pathlib import Path
import pyarrow.parquet as pq

month = 1
year = 2023
path = Path("..") / "data" / "processed" / f"rides_{year}_{month:02}.parquet"

table = pq.read_table(path)
rides = table.to_pandas()
rides.iloc[1000:1020]


,pickup_datetime,pickup_location_id
1000,2023-01-06 19:26:48,JC115
1001,2023-01-06 19:27:07,JC115
1002,2023-01-28 12:28:52,JC115
1003,2023-01-26 21:33:07,JC115
1004,2023-01-08 14:44:36,JC115
1005,2023-01-28 12:28:09,JC115
1006,2023-01-23 21:48:01,JC115
1007,2023-01-04 12:19:13,JC115
1008,2023-01-08 16:08:27,JC115
1009,2023-01-08 16:08:40,JC115


In [3]:
rides.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7246 entries, 0 to 7245
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   pickup_datetime     7246 non-null   datetime64[ns]
 1   pickup_location_id  7246 non-null   object        
dtypes: datetime64[ns](1), object(1)
memory usage: 113.3+ KB


In [4]:
rides["pickup_hour"] = rides["pickup_datetime"].dt.floor('h')
rides.tail()


,pickup_datetime,pickup_location_id,pickup_hour
7241,2023-01-08 02:16:19,HB103,2023-01-08 02:00:00
7242,2023-01-21 15:32:18,HB103,2023-01-21 15:00:00
7243,2023-01-29 14:01:15,HB103,2023-01-29 14:00:00
7244,2023-01-30 08:28:41,HB103,2023-01-30 08:00:00
7245,2023-01-28 17:53:21,HB103,2023-01-28 17:00:00


In [5]:
agg_rides = rides.groupby(["pickup_hour", "pickup_location_id"]).size().reset_index()
agg_rides.rename(columns={0: "rides"}, inplace=True)
agg_rides.tail()


,pickup_hour,pickup_location_id,rides
1578,2023-01-31 21:00:00,HB103,3
1579,2023-01-31 21:00:00,JC115,8
1580,2023-01-31 22:00:00,HB103,1
1581,2023-01-31 22:00:00,JC115,2
1582,2023-01-31 23:00:00,JC115,6


In [6]:
# Get the full range of hourly timestamps
hour_col = "pickup_hour"
full_hours = pd.date_range(
    start=agg_rides[hour_col].min(),
    end=agg_rides[hour_col].max(),
    freq="h"
)

full_hours


DatetimeIndex(['2023-01-01 00:00:00', '2023-01-01 01:00:00',
               '2023-01-01 02:00:00', '2023-01-01 03:00:00',
               '2023-01-01 04:00:00', '2023-01-01 05:00:00',
               '2023-01-01 06:00:00', '2023-01-01 07:00:00',
               '2023-01-01 08:00:00', '2023-01-01 09:00:00',
               ...
               '2023-01-31 14:00:00', '2023-01-31 15:00:00',
               '2023-01-31 16:00:00', '2023-01-31 17:00:00',
               '2023-01-31 18:00:00', '2023-01-31 19:00:00',
               '2023-01-31 20:00:00', '2023-01-31 21:00:00',
               '2023-01-31 22:00:00', '2023-01-31 23:00:00'],
              dtype='datetime64[ns]', length=744, freq='H')

In [7]:
# Unique location IDs in the dataset
location_col = "pickup_location_id"
all_locations = agg_rides[location_col].unique()
all_locations


array(['HB102', 'HB103', 'JC115'], dtype=object)

In [8]:
# Create a list to store the complete data
complete_data = []
rides_col = "rides"

# Iterate over all combinations of hours and locations
for hour in full_hours:
    for location in all_locations:
        # Check if the combination exists in the original DataFrame
        subset = agg_rides[(agg_rides[hour_col] == hour) & (agg_rides[location_col] == location)]
        if not subset.empty:
            # If it exists, append the row
            complete_data.append(subset.iloc[0].to_dict())
        else:
            # If missing, append a row with 0 rides
            complete_data.append({hour_col: hour, location_col: location, rides_col: 0})

df_complete = pd.DataFrame(complete_data)
df_complete


,pickup_hour,pickup_location_id,rides
0,2023-01-01 00:00:00,HB102,2
1,2023-01-01 00:00:00,HB103,0
2,2023-01-01 00:00:00,JC115,0
3,2023-01-01 01:00:00,HB102,8
4,2023-01-01 01:00:00,HB103,6
...,...,...,...
2227,2023-01-31 22:00:00,HB103,1
2228,2023-01-31 22:00:00,JC115,2
2229,2023-01-31 23:00:00,HB102,0
2230,2023-01-31 23:00:00,HB103,0


In [9]:
df_complete_sorted = df_complete.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)
df_complete_sorted.head()


,pickup_hour,pickup_location_id,rides
0,2023-01-01 00:00:00,HB102,2
1,2023-01-01 01:00:00,HB102,8
2,2023-01-01 02:00:00,HB102,2
3,2023-01-01 03:00:00,HB102,1
4,2023-01-01 04:00:00,HB102,1


In [10]:
# Create full combinations of hours and locations
full_combinations = pd.DataFrame(
    [(hour, location) for hour in full_hours for location in all_locations],
    columns=["pickup_hour", "pickup_location_id"]
)

full_combinations.head()


,pickup_hour,pickup_location_id
0,2023-01-01 00:00:00,HB102
1,2023-01-01 00:00:00,HB103
2,2023-01-01 00:00:00,JC115
3,2023-01-01 01:00:00,HB102
4,2023-01-01 01:00:00,HB103


In [11]:
agg_rides.head()

,pickup_hour,pickup_location_id,rides
0,2023-01-01 00:00:00,HB102,2
1,2023-01-01 01:00:00,HB102,8
2,2023-01-01 01:00:00,HB103,6
3,2023-01-01 01:00:00,JC115,1
4,2023-01-01 02:00:00,HB102,2


In [12]:
merged_df = pd.merge(full_combinations, agg_rides, on=["pickup_hour", "pickup_location_id"], how="left")
merged_df.head()


,pickup_hour,pickup_location_id,rides
0,2023-01-01 00:00:00,HB102,2.0
1,2023-01-01 00:00:00,HB103,NaN
2,2023-01-01 00:00:00,JC115,NaN
3,2023-01-01 01:00:00,HB102,8.0
4,2023-01-01 01:00:00,HB103,6.0


In [13]:
merged_df["rides"] = merged_df["rides"].fillna(0).astype(int)
merged_df.head()


,pickup_hour,pickup_location_id,rides
0,2023-01-01 00:00:00,HB102,2
1,2023-01-01 00:00:00,HB103,0
2,2023-01-01 00:00:00,JC115,0
3,2023-01-01 01:00:00,HB102,8
4,2023-01-01 01:00:00,HB103,6


In [14]:
def fill_missing_rides_full_range(df, hour_col, location_col, rides_col):
    """
    Fills in missing rides for all hours in the range and all unique locations.

    Parameters:
    - df: DataFrame with columns [hour_col, location_col, rides_col]
    - hour_col: Name of the column containing hourly timestamps
    - location_col: Name of the column containing location IDs
    - rides_col: Name of the column containing ride counts

    Returns:
    - DataFrame with missing hours and locations filled in with 0 rides
    """
    df[hour_col] = pd.to_datetime(df[hour_col])
    full_hours = pd.date_range(start=df[hour_col].min(), end=df[hour_col].max(), freq="h")
    all_locations = df[location_col].unique()
    full_combinations = pd.DataFrame(
        [(hour, location) for hour in full_hours for location in all_locations],
        columns=[hour_col, location_col]
    )
    merged_df = pd.merge(full_combinations, df, on=[hour_col, location_col], how='left')
    merged_df[rides_col] = merged_df[rides_col].fillna(0).astype(int)
    return merged_df


In [15]:
agg_rides

,pickup_hour,pickup_location_id,rides
0,2023-01-01 00:00:00,HB102,2
1,2023-01-01 01:00:00,HB102,8
2,2023-01-01 01:00:00,HB103,6
3,2023-01-01 01:00:00,JC115,1
4,2023-01-01 02:00:00,HB102,2
...,...,...,...
1578,2023-01-31 21:00:00,HB103,3
1579,2023-01-31 21:00:00,JC115,8
1580,2023-01-31 22:00:00,HB103,1
1581,2023-01-31 22:00:00,JC115,2


In [16]:
hour_col = "pickup_hour"
location_col = "pickup_location_id"
rides_col = "rides"

agg_data_filled = fill_missing_rides_full_range(
    agg_rides, hour_col, location_col, rides_col
).sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)


In [17]:
pd.set_option('display.max_rows', 2000)
agg_data_filled.head(1489)


,pickup_hour,pickup_location_id,rides
0,2023-01-01 00:00:00,HB102,2
1,2023-01-01 01:00:00,HB102,8
2,2023-01-01 02:00:00,HB102,2
3,2023-01-01 03:00:00,HB102,1
4,2023-01-01 04:00:00,HB102,1
5,2023-01-01 05:00:00,HB102,0
6,2023-01-01 06:00:00,HB102,0
7,2023-01-01 07:00:00,HB102,0
8,2023-01-01 08:00:00,HB102,0
9,2023-01-01 09:00:00,HB102,0


In [18]:
from typing import Optional, List
import plotly.express as px
import pandas as pd

def plot_rides(
    rides: pd.DataFrame,
    locations: Optional[List[int]] = None
):

    rides_to_plot = rides[rides.pickup_location_id.isin(locations)] if locations else rides

    fig = px.line(
        rides_to_plot,
        x="pickup_hour",
        y="rides",
        color="pickup_location_id",
        template="none"
    )

    fig.show()


In [19]:
plot_rides(agg_data_filled, locations=["JC115", "HB102", "HB103"])


In [19]:
from pathlib import Path

month = 1
year = 2023
path = Path("..") / "data" / "processed" / f"ts_data_{year}_{month:02}.parquet"

agg_data_filled.to_parquet(path, engine="pyarrow", index=False)

print(f"Saved: {path}")


Saved: ..\data\processed\ts_data_2023_01.parquet
